In [66]:
import pandas as pd
from sqlite3 import connect
import xlwings as xw
import numpy as np
from datetime import datetime as dt
import os
import pathlib
import sources.parts_list_0802 as pl

In [2]:
conn = connect('quotation.db')

In [55]:
def working_day(start_date,end_date):
    try:
        res = np.busday_count(start_date.strftime('%Y-%m-%d'),
                          start_date.strftime('%Y-%m-%d'))
    except:
        res = np.busday_count(start_date,end_date)
    return res +1 
conn.create_function("WORKING_DAY",2,working_day)

In [28]:
working_day('2023-10-03','2023-10-11')

6

In [33]:
q = "SELECT WORKING_DAY('2023-10-03','2023-10-11')"
pd.read_sql(q,conn)

,"WORKING_DAY('2023-10-03','2023-10-11')"
0,b'\x06\x00\x00\x00\x00\x00\x00\x00'


In [60]:
# new format
q = f'''
        SELECT DISTINCT c.[rma no.] as [ExFM RMA#],
        strftime('%Y-%m-%d',recieve_date) as [Receive],
        strftime('%Y-%m-%d',R.[start time]) as [Start_date],
        strftime('%Y-%m-%d',c.qc) as [End_date],
        '' as [Repair TAT],
        repair_status,
        [repair user name] AS repair_user,
        customer_name,model,serial_no,
        [defect note] AS [Problem Descript.],
        '' AS Issue,
        strftime('%Y-%m-%d',[Part Select Date]) AS Quote_date,
        strftime('%Y-%m-%d',AUTHORIZED_DATE) AS Confirm_date,
        '' AS [PO TAT],
        '' AS [Part Order Date],
        '' AS [Part Receive Date],
        '' [Parts TAT],
        '' AS [Total TAT]
        
        FROM consolidated c
        LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.]
    '''
try:
    hcap = pd.read_sql(q,conn)
    display(hcap)
except Exception as e:
    print(e)

,ExFM RMA#,Receive,Start_date,End_date,Repair TAT,REPAIR_STATUS,repair_user,CUSTOMER_NAME,MODEL,SERIAL_NO,Problem Descript.,Issue,Quote_date,Confirm_date,PO TAT,Part Order Date,Part Receive Date,Parts TAT,Total TAT
0,FMSV2023030075,2023-03-30,2023-10-03,2023-10-11,,Shipped,Nguyen Tuan Minh,"FUJIFILM Vietnam Co., Ltd.",EC-530WI,1C603K041,1.Stain image\r\n2.FCT scratch\r\n3.BSA stiff,,2023-10-03,2023-10-03,,,,,
1,FMSV2023090043,2023-09-20,2023-09-20,2023-10-11,,Shipped,Nguyen Tuan Minh,"FUJIFILM Vietnam Co., Ltd.",EG-740N,2G399K002,None,,2023-09-20,2023-09-20,,,,,
2,FMSV2023090044,2023-09-20,2023-09-20,2023-10-11,,Shipped,Nguyen Tuan Minh,"FUJIFILM Vietnam Co., Ltd.",EC-760R-V/I,6C728K116,None,,2023-09-20,2023-09-20,,,,,
3,FMSV2023090058,2023-09-26,2023-09-29,2023-10-11,,Shipped,Nguyen Tuan Minh,"FUJIFILM Vietnam Co., Ltd.",EP-6000,1V696K004,None,,2023-09-29,2023-09-29,,,,,
4,FMSV2023060027,2023-06-27,2023-10-10,2023-10-12,,Shipped,Nguyen Thai,FTYO - Loaner Asset,ED-530XT8,3D103L050,FCT leaked,,2023-10-10,2023-10-10,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,FMSV2023100030,2023-10-19,None,None,,Receive,None,Viet Nhat Clinic 3 - Dr.Giang'S Clinic,EG-530UR2,RU039A048,None,,None,None,,,,,
373,FMSV2023100031,2023-10-23,None,None,,Receive,None,National Lung Hospital,EB-530T,1B084K400,None,,None,None,,,,,
374,FMSV2023100032,2023-10-23,None,None,,Receive,None,Viet Duc Hospital,EC-760R-V/I,8C728K200,None,,None,None,,,,,
375,FMSV2023100033,2023-10-23,None,None,,Receive,None,Viet Duc Hospital,EC-760R-V/I,8C728K201,None,,None,None,,,,,


In [58]:
#calculate TAT
for i in range(len(hcap)):
    # TAT repair
    if hcap['Start_date'][i] is not None and hcap['End_date'][i] is not None:
        hcap['Repair TAT'][i] = working_day(hcap['Start_date'][i],hcap['End_date'][i])
    # PO TAT
    if hcap['Quote_date'][i] is not None and hcap['Confirm_date'][i] is not None:
        hcap['PO TAT'][i] = working_day(hcap['Quote_date'][i],hcap['Confirm_date'][i])
        
hcap

,ExFM RMA#,Receive,Start_date,End_date,Repair TAT,REPAIR_STATUS,repair_user,CUSTOMER_NAME,MODEL,SERIAL_NO,Problem Descript.,Issue,Quote_date,Confirm_date,PO TAT,Part Order Date,Part Receive Date,Parts TAT,Total TAT
0,FMSV2023030075,2023-03-30,2023-10-03,2023-10-11,7,Shipped,Nguyen Tuan Minh,"FUJIFILM Vietnam Co., Ltd.",EC-530WI,1C603K041,1.Stain image\r\n2.FCT scratch\r\n3.BSA stiff,,2023-10-03,2023-10-03,1,,,,
1,FMSV2023090043,2023-09-20,2023-09-20,2023-10-11,16,Shipped,Nguyen Tuan Minh,"FUJIFILM Vietnam Co., Ltd.",EG-740N,2G399K002,None,,2023-09-20,2023-09-20,1,,,,
2,FMSV2023090044,2023-09-20,2023-09-20,2023-10-11,16,Shipped,Nguyen Tuan Minh,"FUJIFILM Vietnam Co., Ltd.",EC-760R-V/I,6C728K116,None,,2023-09-20,2023-09-20,1,,,,
3,FMSV2023090058,2023-09-26,2023-09-29,2023-10-11,9,Shipped,Nguyen Tuan Minh,"FUJIFILM Vietnam Co., Ltd.",EP-6000,1V696K004,None,,2023-09-29,2023-09-29,1,,,,
4,FMSV2023060027,2023-06-27,2023-10-10,2023-10-12,3,Shipped,Nguyen Thai,FTYO - Loaner Asset,ED-530XT8,3D103L050,FCT leaked,,2023-10-10,2023-10-10,1,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,FMSV2023100030,2023-10-19,None,None,,Receive,None,Viet Nhat Clinic 3 - Dr.Giang'S Clinic,EG-530UR2,RU039A048,None,,None,None,,,,,
373,FMSV2023100031,2023-10-23,None,None,,Receive,None,National Lung Hospital,EB-530T,1B084K400,None,,None,None,,,,,
374,FMSV2023100032,2023-10-23,None,None,,Receive,None,Viet Duc Hospital,EC-760R-V/I,8C728K200,None,,None,None,,,,,
375,FMSV2023100033,2023-10-23,None,None,,Receive,None,Viet Duc Hospital,EC-760R-V/I,8C728K201,None,,None,None,,,,,


In [63]:
today = dt.now().strftime('%y%d%m')
folder_name = 'inspection'
path = pathlib.Path().absolute()
file_name = f'FFVN-Repair_{today}.xlsx'
file_name = os.path.join(path,folder_name,file_name)
hcap.to_excel(file_name,index = False)

In [64]:
xw.Book(file_name)

<Book [FFVN-Repair_232410.xlsx]>